Agregamos primero importe de la librerias necesaria

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import re


Unificacion de archivos csv en un dataframe para realizar un analisis completo, asi como validar y unificar titulos y tipos de datos

In [ ]:
# Función para realizar el análisis exploratorio
def analizar_dataframe(df, nombre):
    print(f"--- Análisis del DataFrame: {nombre} ---\n")
    
    # 1. Mostrar las primeras filas
    print("Primeras filas:")
    print(df.head())
    print("\n")
    
    # 2. Revisar la estructura
    print("Información del DataFrame:")
    print(df.info())
    print("\n")
    
    # 3. Verificar valores nulos
    print("Valores nulos por columna:")
    print(df.isnull().sum())
    print("\n")
    
    # 4. Estadísticas descriptivas
    print("Estadísticas descriptivas:")
    print(df.describe(include='all'))
    print("\n")
    
    # 5. Visualizar la distribución de las variables numéricas
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        plt.figure(figsize=(10, 6))
        sns.histplot(df[col], kde=True, bins=30)
        plt.title(f'Distribución de {col}')
        plt.xlabel(col)
        plt.ylabel('Frecuencia')
        plt.show()
    
    # 6. Identificar valores atípicos
    for col in numeric_cols:
        plt.figure(figsize=(10, 6))
        sns.boxplot(x=df[col])
        plt.title(f'Boxplot de {col}')
        plt.xlabel(col)
        plt.show()
    
    # 7. Graficar precio vs modelo utilizando boxplot
    if 'Selling_Price' in df.columns and 'Car_Name' in df.columns:
        plt.figure(figsize=(12, 6))
        sns.boxplot(x='Car_Name', y='Selling_Price', data=df)
        plt.title('Precio de Venta vs Modelo de Coche')
        plt.xticks(rotation=45)
        plt.xlabel('Modelo de Coche')
        plt.ylabel('Precio de Venta')
        plt.show()
    
    # 8. Graficar precio promedio vs modelo utilizando barplot
    if 'Selling_Price' in df.columns:
        plt.figure(figsize=(12, 6))
        avg_price = df.groupby('Car_Name')['Selling_Price'].mean().reset_index()
        sns.barplot(x='Car_Name', y='Selling_Price', data=avg_price, palette='viridis')
        plt.title('Precio Promedio de Venta vs Modelo de Coche')
        plt.xticks(rotation=45)
        plt.xlabel('Modelo de Coche')
        plt.ylabel('Precio Promedio de Venta')
        plt.show()
    
    # 9. Graficar precio promedio vs año utilizando barplot
    if 'Year' in df.columns and 'Selling_Price' in df.columns:
        plt.figure(figsize=(12, 6))
        avg_price_by_year = df.groupby('Year')['Selling_Price'].mean().reset_index()
        sns.barplot(x='Year', y='Selling_Price', data=avg_price_by_year, palette='viridis')
        plt.title('Precio Promedio de Venta vs Año del Modelo')
        plt.xlabel('Año del Modelo')
        plt.ylabel('Precio Promedio de Venta')
        plt.show()

# Cargar los archivos
df1 = pd.read_csv("Data\\car data.csv")
df2 = pd.read_csv("Data\\CAR DETAILS FROM CAR DEKHO.csv")
df3 = pd.read_csv("Data\\Car details v3.csv")
df4 = pd.read_csv("Data\\car details v4.csv")

# Analizar cada DataFrame
analizar_dataframe(df1, "car data.csv")
analizar_dataframe(df2, "CAR DETAILS FROM CAR DEKHO.csv")
analizar_dataframe(df3, "Car details v3.csv")
analizar_dataframe(df4, "car details v4.csv")


Validacion de valores nulos y estadisticas

In [9]:
# Verificar valores nulos y estadísticas
def verificar_nulos_y_estadisticas(df):
    print("Valores nulos por columna:")
    print(df.isnull().sum())
    print("\nEstadísticas descriptivas:")
    print(df.describe(include='all'))
    print("\n")

# Añadir esta línea en cada análisis individual
verificar_nulos_y_estadisticas(df1)
verificar_nulos_y_estadisticas(df2)
verificar_nulos_y_estadisticas(df3)
verificar_nulos_y_estadisticas(df4)

Valores nulos por columna:
Car_Name         0
Year             0
Selling_Price    0
Present_Price    0
Kms_Driven       0
Fuel_Type        0
Seller_Type      0
Transmission     0
Owner            0
dtype: int64

Estadísticas descriptivas:
       Car_Name         Year  Selling_Price  Present_Price     Kms_Driven  \
count       301   301.000000     301.000000     301.000000     301.000000   
unique       98          NaN            NaN            NaN            NaN   
top        city          NaN            NaN            NaN            NaN   
freq         26          NaN            NaN            NaN            NaN   
mean        NaN  2013.627907       4.661296       7.628472   36947.205980   
std         NaN     2.891554       5.082812       8.644115   38886.883882   
min         NaN  2003.000000       0.100000       0.320000     500.000000   
25%         NaN  2012.000000       0.900000       1.200000   15000.000000   
50%         NaN  2014.000000       3.600000       6.400000   32000.0

Identificando lo anterior se encuentra que el precio de venta es una variable util como variable objetivo para su analisis y prediccion y creacion del modelo de regresion lineal

In [ ]:
# Cargar el archivo CSV
try:
    df = pd.read_csv("Data\\car details v4.csv", delimiter=',', quotechar='"')
    print(f"Forma del DataFrame: {df.shape}")
except Exception as e:
    print(f"Ocurrió un error al importar el archivo: {e}")
# Mostrar las primeras filas
print(df.info())
print("\n")

Forma del DataFrame: (2059, 20)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2059 entries, 0 to 2058
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Make                2059 non-null   object 
 1   Model               2059 non-null   object 
 2   Price               2059 non-null   int64  
 3   Year                2059 non-null   int64  
 4   Kilometer           2059 non-null   int64  
 5   Fuel Type           2059 non-null   object 
 6   Transmission        2059 non-null   object 
 7   Location            2059 non-null   object 
 8   Color               2059 non-null   object 
 9   Owner               2059 non-null   object 
 10  Seller Type         2059 non-null   object 
 11  Engine              1979 non-null   object 
 12  Max Power           1979 non-null   object 
 13  Max Torque          1979 non-null   object 
 14  Drivetrain          1923 non-null   object 
 15  Length              199

Con el comando anterior se valida el tipo de variables con el que trabajamos, Se realiza una validacion a continuacion con la descripcion estadistica de sus variables, percentiles meadia y demas.

In [ ]:
print("Descripcion del dataframe seleccionado" "\n")
print(df.describe())

descripcion del dataframe seleccionado

              Price         Year     Kilometer       Length        Width  \
count  2.059000e+03  2059.000000  2.059000e+03  1995.000000  1995.000000   
mean   1.702992e+06  2016.425449  5.422471e+04  4280.860652  1767.991980   
std    2.419881e+06     3.363564  5.736172e+04   442.458507   135.265825   
min    4.900000e+04  1988.000000  0.000000e+00  3099.000000  1475.000000   
25%    4.849990e+05  2014.000000  2.900000e+04  3985.000000  1695.000000   
50%    8.250000e+05  2017.000000  5.000000e+04  4370.000000  1770.000000   
75%    1.925000e+06  2019.000000  7.200000e+04  4629.000000  1831.500000   
max    3.500000e+07  2022.000000  2.000000e+06  5569.000000  2220.000000   

            Height  Seating Capacity  Fuel Tank Capacity  
count  1995.000000       1995.000000         1946.000000  
mean   1591.735338          5.306266           52.002210  
std     136.073956          0.822170           15.110198  
min    1165.000000          2.000000   

Se valida los valores Nulos para identificar si las variables a usar se encuentran comprometidas y requieren ajustes a nivel de eliminarlos o normarlizarlos segun el caso.

In [22]:
# Contar valores nulos por columna
print(df.isnull().sum())


Price                0
Year                 0
Kilometer            0
Engine              80
Max Power           80
Length              64
Width               64
Height              64
Seating Capacity    64
Fuel Type            0
Transmission         0
Location             0
dtype: int64


Limpieza y Preprocesamiento de Datos en las columnas clave para el ejercicio de regresion lineal, es decir las variables que vamos a usar como dependiente e independientes.

In [24]:
def extract_numeric(value):
    # Extrae números de una cadena usando expresiones regulares
    if isinstance(value, str):
        # Busca el primer número en la cadena
        result = re.findall(r'\d+', value)
        return float(result[0]) if result else np.nan
    return np.nan


In [25]:
# Limpiar y convertir las columnas relevantes
df['Max Power'] = df['Max Power'].apply(extract_numeric)
df['Max Torque'] = df['Max Torque'].apply(extract_numeric)
df['Engine'] = df['Engine'].apply(extract_numeric)


NameError: name 're' is not defined

In [23]:
# Imputar valores nulos en columnas numéricas
df['Engine'] = df['Engine'].fillna(df['Engine'].median())
df['Max Power'] = df['Max Power'].fillna(df['Max Power'].median())
df['Length'] = df['Length'].fillna(df['Length'].median())
df['Width'] = df['Width'].fillna(df['Width'].median())
df['Height'] = df['Height'].fillna(df['Height'].median())
df['Seating Capacity'] = df['Seating Capacity'].fillna(df['Seating Capacity'].median())
df['Fuel Tank Capacity'] = df['Fuel Tank Capacity'].fillna(df['Fuel Tank Capacity'].median())

TypeError: Cannot convert ['87 bhp @ 6000 rpm' '74 bhp @ 4000 rpm' '79 bhp @ 6000 rpm' ...
 '70 bhp @ 6250 rpm' '188 bhp @ 4000 rpm' '70 bhp @ 3600 rpm'] to numeric